In [ ]:
import json
import pandas as pd
import uuid

from flask import Flask, Response, request, jsonify

### Helper functies

In [ ]:
def get_json_response(body, status_code):
    response = Response(
            response=json.dumps(body)
            ,status=status_code
            ,mimetype='application/json'
        )
    return response

### Database
Hier kan je een kijkje nemen in de data die beschikbaar is. Deze tabel kan je er bij houden tijdens het ontwikkelen van je API om te testen of bepaalde acties het gewenste resultaat bereiken. Ook kan je de tabel gebruiken om voorbeeld ID's er uit te halen om te testen. Er staat een kopie van `employees` in de root van dit project. Kopieer deze in je data map als je fouten maakt tijdens de laatste opdracht (3).

In [ ]:
employees = pd.read_csv('data/employees.csv')
employees

### API definitie
Hieronder zie je het raamwerk van de API die we gaan bouwen. Veel is al geïmplementeerd, dus dit hoeven we niet meer zelf te doen.
De inhoud echter van elk request type is nog ongedefinieerd en moeten we zelf gaan bouwen. Het is belangrijk dat bij elke response de functie `get_json_response()` wordt gebruikt, waarbij de juiste HTML status code wordt meegegeven (zie: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status). Je staat vrij om de API te implementeren zoals je zelf wil, zo lang je API maar geen errors teruggeeft. Het is aan te raden om het niet te ingewikkeld te maken.

De volgende stukken moeten worden geïmplementeerd:
1. Bij `get_employee_by_id()`: als een ID wordt opgevraagd die niet bestaat, wat zou de API dan terug moeten geven?
2. Bij `list_employees()`: geef een json array terug met alle werknemers in de database.
3. Bij `add_employee()`: zorg dat er op basis van de juiste body in de POST request een werknemer wordt toegevoegd in de database. Let op: de ID wordt automatisch gegenereerd met de functie `uuid.uuid4()` Deze functie geeft dus geen json response terug, maar doet een bewerking op de database.

In [ ]:
app = Flask(__name__)

@app.route('/', methods=['GET'])
def get_info():
    response = get_json_response('Welcome at the D-Data Human Resource API!', status_code=200)
    return response


@app.route('/employee/<id>', methods=['GET'])
def get_employee_by_id(id):
    employees = pd.read_csv('data/employees.csv')
    employee = employees.query('id == @id')
    
    if len(employee) == 0:
        response = get_json_response('ID not in database', status_code=404)
        return response

    json_body = employee.to_dict(orient='records')[0]
    response = get_json_response(json_body, status_code=200)

    return response


@app.route('/employee', methods=['GET'])
def list_employees():
    employees = pd.read_csv('data/employees.csv')
    
    json_body = employees.to_dict(orient='records')
    response = get_json_response(json_body, status_code=200)
    return response
    

@app.route('/employee', methods=['POST'])
def add_employee():
    body = json.loads(request.get_json())
    name = body.get('name')
    role = body.get('role')

    employees = pd.read_csv('data/employees.csv')
    
    employee = {
        'id': uuid.uuid4(),
        'name': name,
        'role': role
    }

    employees = employees._append(employee, ignore_index=True)
    employees.to_csv('data/employees.csv', index=False)

    return get_json_response('Employee succesfully added', status_code=200)


if __name__ == '__main__':
    app.run(debug=False, host='0.0.0.0')
